In [206]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [207]:
df = pd.read_csv("CHALLENGER_Timeline.csv")


In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323310 entries, 0 to 323309
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   matchID             323310 non-null  object
 1   timestamp           323310 non-null  int64 
 2   puuid               323310 non-null  object
 3   teamID              323310 non-null  int64 
 4   ID                  323310 non-null  int64 
 5   win                 323310 non-null  bool  
 6   championId          323310 non-null  int64 
 7   championName        323310 non-null  object
 8   TeamPosition        293597 non-null  object
 9   individualPosition  323310 non-null  object
 10  totalGold           323310 non-null  int64 
 11  currentGold         323310 non-null  int64 
 12  xp                  323310 non-null  int64 
 13  level               323310 non-null  int64 
 14  dragon              323310 non-null  int64 
 15  horde               323310 non-null  int64 
 16  ri

In [209]:
df["Tier"] = "challenger"

In [210]:
df["win"] = np.where(df["win"] == True, 1, 0)

In [211]:
df["timestamp_min"] = df["timestamp"]//60000

In [212]:
df.drop_duplicates(inplace = True)
print(df.duplicated().sum())
df = df.sort_values(by = "win")
df = df.reset_index(drop = True)
df

0


,matchID,timestamp,puuid,teamID,ID,win,championId,championName,TeamPosition,individualPosition,totalGold,currentGold,xp,level,dragon,horde,riftherald,baron_nasher,timestamp_min
0,TW2_208865923,1560472,b1SDdlsouQMAZGTIoGeKPBBeJXGysCVv9qeF6VwUg7Hf0J...,100,5,0,59,JarvanIV,JUNGLE,JUNGLE,8994,474,10354,13,2,0,0,0,26
1,KR_7130709343,180080,xZVxv-cfKvReNfXaV3loih3kXo0U1UVePZjua96e_3Vb1h...,200,9,0,145,Kaisa,BOTTOM,BOTTOM,867,367,461,2,0,0,0,0,3
2,KR_7130709343,180080,NH9rqXA_6Ze7fGfd_6yN3LFVcpJa25u5780b1Gx_W9QS5x...,200,10,0,111,Nautilus,UTILITY,UTILITY,684,184,461,2,0,0,0,0,3
3,KR_7130709343,240089,GUqtYpQitUYtYJjJreHjLkhvcMoNupdNzVZF7ekwhGk-jx...,200,6,0,122,Darius,TOP,TOP,1204,704,1477,4,0,0,0,0,4
4,KR_7130709343,240089,RKsJd936cmrf77MsZDQar5Y7F66oUylVkiaOyP1XZHE6I5...,200,7,0,121,Khazix,JUNGLE,JUNGLE,1255,330,1011,3,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323305,KR_6827979095,540187,ldzVfLi0CGuWZvR3hjUzn99__PHLxvRheVlsYYhfIX0f5t...,100,5,1,53,Blitzcrank,UTILITY,UTILITY,2767,692,2278,5,0,0,0,0,9
323306,TW2_226286289,0,jsPTYZBhY_nQrRGUbPnAl2YBd159nsa0DNxR-ycPYWi31R...,100,5,1,9,FiddleSticks,UTILITY,UTILITY,500,500,0,1,0,0,0,0,0
323307,TW2_226286289,0,ELU5fRxdB_jq2mze4ukYh_8me4HprAudbUmK015ABATbdE...,100,4,1,221,Zeri,BOTTOM,BOTTOM,500,500,0,1,0,0,0,0,0
323308,TW2_226286289,0,V0GJavSXO_pqY_wled8nhxHC-3792H5V8g4IjZLHGMDab5...,100,2,1,141,Kayn,JUNGLE,JUNGLE,500,500,0,1,0,0,0,0,0


In [213]:
for column in ["dragon", "horde", "riftherald", "baron_nasher"]:
    df[f"Team{column}"] = df["matchID"].astype(str) + " " + df["timestamp"].astype(str) + " " + df["teamID"].astype(str)
    group_df = df.groupby(f"Team{column}")[column].sum()
    group_df_dict = group_df.to_dict()
    df[f"Team{column}"] = df[f"Team{column}"].map(group_df_dict)
    df[f"diff_{column}_from_lose"] = df[f"Team{column}"] - df.groupby(["matchID", "timestamp", "TeamPosition"])[f"Team{column}"].transform(lambda x : x.iloc[0])

In [214]:
# for column in ["dragon", "horde", "riftherald", "baron_nasher"]:
#     df[f"diff_{column}_from_lose"] = df[f"Team{column}"] - df.groupby(["matchID", "timestamp", "TeamPosition"])[f"Team{column}"].transform(lambda x : x.iloc[0])

In [215]:
for column in ["dragon", "horde", "riftherald", "baron_nasher"]:
    df[f"Diff_{column}"] = df["matchID"].astype(str) + " " + df["timestamp"].astype(str) + " " + df["TeamPosition"].astype(str)
    group_df = df.groupby(f"Diff_{column}")[f"diff_{column}_from_lose"].sum()
    group_df_dict = group_df.to_dict()
    df[f"Diff_{column}"] = df[f"Diff_{column}"].map(group_df_dict)
    df.loc[df["win"] == 0, f"Diff_{column}"] = df[df["win"] == 0][f"Diff_{column}"]*(-1)

In [216]:
df.groupby(["matchID", "timestamp", "teamID"]).max(numeric_only=True)

ID  win  championId  totalGold  currentGold  \
matchID       timestamp teamID                                                
KR_6496730615 0         100      5    1         523        500          500   
                        200     10    0         777        500          500   
              60031     100      5    1         523        500            0   
                        200     10    0         777        500          500   
              120052    100      5    1         523        652          152   
...                             ..  ...         ...        ...          ...   
TW2_233512902 1740580   200     10    1         901      15712         1239   
              1800608   100      5    0         555      13254          740   
                        200     10    1         901      16680         1736   
              1806473   100      5    0         555      13266          752   
                        200     10    1         901      16692         1747   

                                   xp  level  dragon  horde  riftherald  ...  \
matchID       timestamp teamID                                           ...   
KR_6496730615 0         100         0      1       0      0           0  ...   
                        200         0      1       0      0           0  ...   
              60031     100         0      1       0      0           0  ...   
                        200         0      1       0      0           0  ...   
              120052    100       325      2       0      0           0  ...   
...                               ...    ...     ...    ...         ...  ...   
TW2_233512902 1740580   200     16771     17       2      3           0  ...   
              1800608   100     15489     16       2      0           1  ...   
                        200     17671     17       2      3           0  ...   
              1806473   100     15489     16       2      0           1  ...   
                        200     17671     17       2      3           0  ...   

                                Teamhorde  diff_horde_from_lose  \
matchID       timestamp teamID                                    
KR_6496730615 0         100             0                   0.0   
                        200             0                   0.0   
              60031     100             0                   0.0   
                        200             0                   0.0   
              120052    100             0                   0.0   
...                                   ...                   ...   
TW2_233512902 1740580   200             3                   3.0   
              1800608   100             0                   0.0   
                        200             3                   3.0   
              1806473   100             0                   0.0   
                        200             3                   3.0   

                                Teamriftherald  diff_riftherald_from_lose  \
matchID       timestamp teamID                                              
KR_6496730615 0         100                  0                        0.0   
                        200                  0                        0.0   
              60031     100                  0                        0.0   
                        200                  0                        0.0   
              120052    100                  0                        0.0   
...                                        ...                        ...   
TW2_233512902 1740580   200                  0                       -1.0   
              1800608   100                  1                        0.0   
                        200                  0                       -1.0   
              1806473   100                  1                        0.0   
                        200                  0                       -1.0   

                                Teambaron_nasher  diff_baron_nasher_from_lose  \
matchID 

In [217]:
df["diff_Gold_from_lose"] = df["totalGold"] - df.groupby(["matchID", "timestamp", "TeamPosition"])["totalGold"].transform(lambda x : x.iloc[0])


In [218]:
df["Diff_Gold"] = df["matchID"].astype(str) + " " + df["timestamp"].astype(str) + " " + df["TeamPosition"].astype(str)
group_df = df.groupby("Diff_Gold")["diff_Gold_from_lose"].sum()
group_df_dict = group_df.to_dict()
df["Diff_Gold"] = df["Diff_Gold"].map(group_df_dict)
df.loc[df["win"] == 0, "Diff_Gold"] = df[df["win"] == 0]["Diff_Gold"]*(-1)

In [219]:
df.dropna(inplace=True)
df

,matchID,timestamp,puuid,teamID,ID,win,championId,championName,TeamPosition,individualPosition,...,Teamriftherald,diff_riftherald_from_lose,Teambaron_nasher,diff_baron_nasher_from_lose,Diff_dragon,Diff_horde,Diff_riftherald,Diff_baron_nasher,diff_Gold_from_lose,Diff_Gold
0,TW2_208865923,1560472,b1SDdlsouQMAZGTIoGeKPBBeJXGysCVv9qeF6VwUg7Hf0J...,100,5,0,59,JarvanIV,JUNGLE,JUNGLE,...,0,0.0,0,0.0,-0.0,-4.0,-1.0,-0.0,0.0,-963.0
1,KR_7130709343,180080,xZVxv-cfKvReNfXaV3loih3kXo0U1UVePZjua96e_3Vb1h...,200,9,0,145,Kaisa,BOTTOM,BOTTOM,...,0,0.0,0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-39.0
2,KR_7130709343,180080,NH9rqXA_6Ze7fGfd_6yN3LFVcpJa25u5780b1Gx_W9QS5x...,200,10,0,111,Nautilus,UTILITY,UTILITY,...,0,0.0,0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-50.0
3,KR_7130709343,240089,GUqtYpQitUYtYJjJreHjLkhvcMoNupdNzVZF7ekwhGk-jx...,200,6,0,122,Darius,TOP,TOP,...,0,0.0,0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,154.0
4,KR_7130709343,240089,RKsJd936cmrf77MsZDQar5Y7F66oUylVkiaOyP1XZHE6I5...,200,7,0,121,Khazix,JUNGLE,JUNGLE,...,0,0.0,0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323305,KR_6827979095,540187,ldzVfLi0CGuWZvR3hjUzn99__PHLxvRheVlsYYhfIX0f5t...,100,5,1,53,Blitzcrank,UTILITY,UTILITY,...,0,-1.0,0,0.0,-1.0,0.0,-1.0,0.0,920.0,920.0
323306,TW2_226286289,0,jsPTYZBhY_nQrRGUbPnAl2YBd159nsa0DNxR-ycPYWi31R...,100,5,1,9,FiddleSticks,UTILITY,UTILITY,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
323307,TW2_226286289,0,ELU5fRxdB_jq2mze4ukYh_8me4HprAudbUmK015ABATbdE...,100,4,1,221,Zeri,BOTTOM,BOTTOM,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
323308,TW2_226286289,0,V0GJavSXO_pqY_wled8nhxHC-3792H5V8g4IjZLHGMDab5...,100,2,1,141,Kayn,JUNGLE,JUNGLE,...,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [220]:
idx = df.groupby(["matchID", "win", "TeamPosition"])["timestamp"].idxmax()
df_max = df.loc[idx]


,matchID,timestamp,puuid,teamID,ID,win,championId,championName,TeamPosition,individualPosition,...,Teamriftherald,diff_riftherald_from_lose,Teambaron_nasher,diff_baron_nasher_from_lose,Diff_dragon,Diff_horde,Diff_riftherald,Diff_baron_nasher,diff_Gold_from_lose,Diff_Gold
78609,KR_6496730615,1823031,cDWaxtyZZ4APDCUaIj5aaRBbIqkFuclzsAm9FV_8MN489U...,200,9,0,498,Xayah,BOTTOM,BOTTOM,...,1,0.0,1,0.0,3.0,-0.0,-0.0,1.0,0.0,6133.0
78607,KR_6496730615,1823031,x6xHmme5Zyj0Af6XNg5zFfzyaQYMR0G_TNLonNJxgpjxN8...,200,7,0,59,JarvanIV,JUNGLE,JUNGLE,...,1,0.0,1,0.0,3.0,-0.0,-0.0,1.0,0.0,874.0
78608,KR_6496730615,1823031,t-J_Z2HpUtsA7B_hOATyNdvq07kTwAwFo6NZz4cC9p4XKP...,200,8,0,777,Yone,MIDDLE,MIDDLE,...,1,0.0,1,0.0,3.0,-0.0,-0.0,1.0,0.0,1778.0
78606,KR_6496730615,1823031,JlzbjuDNxbT08Ii-8xXN_LO1y9pgnAoAsUZVOqaOzA0_2n...,200,6,0,54,Malphite,TOP,TOP,...,1,0.0,1,0.0,3.0,-0.0,-0.0,1.0,0.0,-2799.0
78610,KR_6496730615,1823031,gAHRDrWXJgrbxo9WU_xrnWFR7yAfKr1MmQx9kjzQuvqUg4...,200,10,0,201,Braum,UTILITY,UTILITY,...,1,0.0,1,0.0,3.0,-0.0,-0.0,1.0,0.0,-4799.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255063,TW2_233512902,1806473,otl2eseixgGs_QQtGBwprD_SSzF2ABd_8qLJYGGsUgfPIT...,200,9,1,901,Smolder,BOTTOM,BOTTOM,...,0,-1.0,1,1.0,0.0,3.0,-1.0,1.0,-863.0,-863.0
255081,TW2_233512902,1806473,M-PGjL6sqUH-9y20ZGEYOPaJBio4Org5wR7uNOG48QOoMV...,200,7,1,11,MasterYi,JUNGLE,JUNGLE,...,0,-1.0,1,1.0,0.0,3.0,-1.0,1.0,3097.0,3097.0
255082,TW2_233512902,1806473,v7-NPvGVp8lfK17TCinlo9fgU5w9VFrFoAVYKnoYKDQM_X...,200,8,1,238,Zed,MIDDLE,MIDDLE,...,0,-1.0,1,1.0,0.0,3.0,-1.0,1.0,4884.0,4884.0
255080,TW2_233512902,1806473,0G-lczgMwV9VJ2KwhlI2nNkgxCISMFsQTvA1AXrG_s-jJw...,200,6,1,74,Heimerdinger,TOP,TOP,...,0,-1.0,1,1.0,0.0,3.0,-1.0,1.0,-1052.0,-1052.0


In [227]:
df[df["riftherald"] == 2]

,matchID,timestamp,puuid,teamID,ID,win,championId,championName,TeamPosition,individualPosition,...,Teamriftherald,diff_riftherald_from_lose,Teambaron_nasher,diff_baron_nasher_from_lose,Diff_dragon,Diff_horde,Diff_riftherald,Diff_baron_nasher,diff_Gold_from_lose,Diff_Gold
4435,KR_6850314214,1620441,xzG7Ap1FtYAm2kl6xdUu2wW6o61tP1vEl6S7Op7FXU0mQF...,100,2,0,154,Zac,JUNGLE,JUNGLE,...,2,0.0,1,0.0,-3.0,-0.0,2.0,1.0,0.0,2260.0
4440,KR_6850314214,1680447,xzG7Ap1FtYAm2kl6xdUu2wW6o61tP1vEl6S7Op7FXU0mQF...,100,2,0,154,Zac,JUNGLE,JUNGLE,...,2,0.0,1,0.0,-3.0,-0.0,2.0,1.0,0.0,2066.0
4445,KR_6850314214,1740451,xzG7Ap1FtYAm2kl6xdUu2wW6o61tP1vEl6S7Op7FXU0mQF...,100,2,0,154,Zac,JUNGLE,JUNGLE,...,2,0.0,1,0.0,-4.0,-0.0,2.0,1.0,0.0,2051.0
4450,KR_6850314214,1786028,xzG7Ap1FtYAm2kl6xdUu2wW6o61tP1vEl6S7Op7FXU0mQF...,100,2,0,154,Zac,JUNGLE,JUNGLE,...,2,0.0,1,0.0,-4.0,-0.0,2.0,1.0,0.0,1927.0
4506,KR_6850314214,1560417,xzG7Ap1FtYAm2kl6xdUu2wW6o61tP1vEl6S7Op7FXU0mQF...,100,2,0,154,Zac,JUNGLE,JUNGLE,...,2,0.0,1,0.0,-3.0,-0.0,2.0,1.0,0.0,3270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322661,KR_6821697751,1620494,0eUitb8CQQ8P-8sgVrBuDCJ_vguRH3DdQxanK7VXVlnrYZ...,200,7,1,11,MasterYi,JUNGLE,JUNGLE,...,2,2.0,1,1.0,-2.0,0.0,2.0,1.0,5156.0,5156.0
322670,KR_6821697751,1680508,0eUitb8CQQ8P-8sgVrBuDCJ_vguRH3DdQxanK7VXVlnrYZ...,200,7,1,11,MasterYi,JUNGLE,JUNGLE,...,2,2.0,2,2.0,-2.0,0.0,2.0,2.0,5716.0,5716.0
322731,KR_6821697751,1080356,0eUitb8CQQ8P-8sgVrBuDCJ_vguRH3DdQxanK7VXVlnrYZ...,200,7,1,11,MasterYi,JUNGLE,JUNGLE,...,2,2.0,0,0.0,-2.0,0.0,2.0,0.0,2150.0,2150.0
322737,KR_6821697751,1140375,0eUitb8CQQ8P-8sgVrBuDCJ_vguRH3DdQxanK7VXVlnrYZ...,200,7,1,11,MasterYi,JUNGLE,JUNGLE,...,2,2.0,0,0.0,-2.0,0.0,2.0,0.0,2628.0,2628.0


In [221]:
df_max.groupby("matchID").count().max()

timestamp                      10
puuid                          10
teamID                         10
ID                             10
win                            10
championId                     10
championName                   10
TeamPosition                   10
individualPosition             10
totalGold                      10
currentGold                    10
xp                             10
level                          10
dragon                         10
horde                          10
riftherald                     10
baron_nasher                   10
timestamp_min                  10
Teamdragon                     10
diff_dragon_from_lose          10
Teamhorde                      10
diff_horde_from_lose           10
Teamriftherald                 10
diff_riftherald_from_lose      10
Teambaron_nasher               10
diff_baron_nasher_from_lose    10
Diff_dragon                    10
Diff_horde                     10
Diff_riftherald                10
Diff_baron_nas

In [222]:
df.to_csv("Modify_Challenger_timeline.csv")
df_max.to_csv("Summary_CHALLENGER.csv")
